Julia code for Kitaev model.
\section{Kitaev model without symmetry}

In [30]:
function index(N1::Int, N2::Int, c::Int, r::Int, atom)
    #Calculate the index of the atoms in the (r,c) unit cell
    c = (c + N1) % N1 
    r = (r + N2) % N2 
    #println("(",r,",",c,")")
    n = r * N1 + c 
    if atom == 'A'
        ind = 2 * n + 1
    elseif atom == 'B'
        ind = 2 * n + 1 + 1
    else
        return println("ERROR: The atom index should be A or B.")
    return ind
    end
end

function bits(tag::Int,i::Int, N1::Int, N2::Int)
    # Chenck the i-th element of the binary representation of tag
    SiteNum  = N1 * N2 * 2
    mask = 2^(SiteNum - i)
    n = tag & mask
    if n == mask
        return 1
    else
        return 0
    end
end
    
function StateConfig(tag::Int, N1::Int, N2::Int)
    #=The configuration of the state with tag.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers       
    Outputs: binary form of tag. type: 1D array=#
    
    dim = 2^(N1 * N2 * 2)
    if tag >= dim
        return println("Error: The tag is out of range.")
    else
        b = string(tag, base = 2, pad = N1 * N2 * 2)
        return b
    end
end

function flip(tag::Int, i::Int, j::Int, N1::Int, N2::Int)
    #=Flip the spin on i,j site.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers
            i,j: position of spins that are flipped
    Output: The tag of new state, type: int =#
    SiteNum = N1 * N2 * 2
    f = 2^(SiteNum - i) + 2^(SiteNum - j)
    return ntag = xor(tag,f)
end

function KitaevRhom(J::Array{Float64,1}, N1::Int, N2::Int)
    #=Kitaev Hamiltonian on N1 * N2 * 2 lattice
    Inputes: N1, N2: column and row numbers
             J: coupling constants
    Outpus: The Non-zero Hamiltonian element H[i] is in the 
            position (row[i], col[i])
            H, row, col : type: list =#
    SiteNum = N1 * N2 * 2
    dim = 2^SiteNum
    
    numNZero = 0  #count the number of non-zero element
    data = Float64[]
    row = Int[]
    col = Int[]
    for tag = 0: dim-1
        #println("The state numnber is: " , tag)
        temp = Dict{Int, Float64}() # A temporay dict
        
        for r = 0: N2-1
            for c = 0: N1-1
                ind = index(N1, N2, c, r, 'A')
                # Sx Sx
                next = index(N1, N2, c, r, 'B')
                
                ntag = flip(tag, ind, next, N1, N2)
                if ntag in keys(temp)
                    temp[ntag] += -J[1] /4
                else
                    temp[ntag] = -J[1] /4
                end  
                
                # Sy Sy
                next = index(N1, N2, c - 1, r, 'B')
                
                ntag = flip(tag, ind, next, N1, N2) 
                if ntag in keys(temp)
                    temp[ntag] += J[2] * (bits(tag,ind, N1, N2) - 0.5) * (bits(tag,next, N1, N2) - 0.5)
                else
                    temp[ntag] = J[2] * (bits(tag,ind, N1, N2) - 0.5) * (bits(tag,next, N1, N2) - 0.5)
                end
                        
                #Sz Sz
                next = index(N1, N2, c, r - 1, 'B')
                    
                if tag in keys(temp)
                    temp[tag] += - J[3] * (bits(tag,ind, N1, N2) - 0.5) * (bits(tag,next, N1, N2) - 0.5)
                else
                    temp[tag] = - J[3] * (bits(tag,ind, N1, N2) - 0.5)* (bits(tag,next, N1, N2) - 0.5)
                end
            end
        end
                        
        odTemp = sort(collect(keys(temp)))
        for sf in odTemp
            append!(col, [tag + 1])
            append!(row, [sf + 1])
            append!(data,[temp[sf]])
            numNZero += 1
        end
    end
    
    #println("non zero elemens number =" , numNZero)
    #println("Percentage =" , numNZero/(dim * dim) * 100,"%")
    return col, row, data
end

function makeH(col, row, data)
    H = sparse(row, col, data)
    return H
end


makeH (generic function with 1 method)

In [27]:
using SparseArrays
using Arpack

@time begin
    N1 = 3
    N2 = 3
    J = [1.0,1.0,1.0]

    col, row, data = KitaevRhom(J, N1, N2)
    H =  makeH(col, row, data)
    
    e,x = eigs(H, which = :SR)
    println(e/(N1*N2*2))
end


non zero elemens number =4980736
Percentage =0.0072479248046875%
[-0.198493, -0.198493, -0.198493, -0.195481, -0.195481, -0.195481]
 12.590909 seconds (17.57 M allocations: 2.170 GiB, 3.07% gc time)


In [33]:
using BenchmarkTools
N1 = 2
N2 = 3
J = [1.0,1.0,1.0]
@benchmark KitaevRhom(J, N1, N2)

BenchmarkTools.Trial: 
  memory estimate:  27.06 MiB
  allocs estimate:  200753
  --------------
  minimum time:     14.660 ms (6.30% GC)
  median time:      16.025 ms (6.42% GC)
  mean time:        16.399 ms (7.92% GC)
  maximum time:     32.543 ms (9.13% GC)
  --------------
  samples:          305
  evals/sample:     1

\section{Kitaev model with translation symmetry}

In [96]:
function index(N1::Int, N2::Int, c::Int, r::Int, atom::Int)
    #Calculate the index of the atoms in the (r,c) unit cell
    #A-sublattice atom = 0; B-subkattice: atom = 1
    c = (c + N1) % N1 
    r = (r + N2) % N2 
    #println("(",r,",",c,")")
    n = r * N1 + c
    ind = 2 * n + 1 + atom
    return ind
end

function bits(tag::Int,i::Int, N1::Int, N2::Int)
    # Chenck the i-th element of the binary representation of tag
    SiteNum  = N1 * N2 * 2
    mask = 2^(SiteNum - i)
    n = tag & mask
    if n == mask
        return 1
    else
        return 0
    end
end

function StateConfig(tag::Int, N1::Int, N2::Int)
    #=The configuration of the state with tag.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers       
    Outputs: binary form of tag. type: 1D array=#
    
    dim = 2^(N1 * N2 * 2)
    if tag >= dim
        return println("Error: The tag is out of range.")
    else
        b = string(tag, base = 2, pad = N1 * N2 * 2)
        return b
    end
end

function flip(tag::Int, i::Int, j::Int, N1::Int, N2::Int)
    #=Flip the spin on i,j site.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers
            i,j: position of spins that are flipped
    Output: The tag of new state, type: int =#
    SiteNum = N1 * N2 * 2
    f = 2^(SiteNum - i) + 2^(SiteNum - j)
    return ntag = xor(tag,f)
end

function cyclebits(tag::Int, n1::Int,n2::Int, N1::Int, N2::Int)
    #=Performs a cyclic permutations of n1 steps to the right and n2 steps upward.
       Inputs: tag: tag of the reference state
               n1: cycle times of n1-direction, columns,if n1=0, no cycle in columns
               n2: cycle times of n2-direction, rows,if n2=0, no cycle in rows
               N1: number of columns
               N2: number of rows
       Outputs: ntag: the tag of the outcoming state=#
    SiteNum = N1 * N2 * 2
    a = reshape(1: 2: SiteNum, N1, N2)
    b = reshape(2: 2: SiteNum, N1, N2)
    a = circshift(a, (n1, n2))
    b = circshift(b, (n1, n2))
    #ncfig = reshape(b, (SiteNum, 1))
    #println(ncfig)
    ntag = 0
    for i = 1: N1*N2
        ntag += 2^(SiteNum - 2*i + 1 ) * bits(tag, a[i], N1, N2)
        ntag += 2^(SiteNum - 2*i) * bits(tag, b[i], N1, N2)
    end  
    return ntag
end

function MomBasis(k1::Int, k2::Int, N1::Int, N2::Int)
    #=Momentum basis list of a given k1, k2
       Inputs: k1: momentum in n1-direction
               k2: momentum in n2-direction
               N1: number of columns
               N2: number of rows
       Outpus: mbasis: momentum basis list
               Ra: list of corresponding cycle period 
                   R1: n1-direction 
                   R2: n2-direction=#
    dim = 2^(N1 * N2 * 2) 
    basis = ones(Int, dim)
    
    m1 = Dict{Int, Int}()
    m2 = Dict{Int, Int}()
    mb = Int[]
    
    #println("Finish initialize")
    # Translation symmetry along n1-direction
    for tag = 0: (dim-1)
        if basis[tag + 1] == 1
            for n1= 1:N1
                ntag = cyclebits(n1, 0, tag, N1, N2)
                if ntag > tag
                    basis[ntag + 1] = 0
                elseif ntag == tag
                    basis[ntag + 1] = 0
                    if k1%(N1/n1) == 0
                        m1[tag] = n1
                    end
                    break
                end
            end
        end
    end
 
    # Translation symmetry along n2-direction
    basis = ones(Int, dim)
    for tag = 0: (dim-1)
        if basis[tag + 1] == 1
            for n2= 1:N2
                ntag = cyclebits(0, n2, tag, N1, N2)
                if ntag > tag
                    basis[ntag + 1] = 0
                elseif ntag == tag
                    basis[ntag + 1] = 0
                    if k2%(N2/n2) == 0
                        m2[tag] = n2
                    end
                    break
                end
            end
        end
    end
   
    for x in collect(keys(m1))
        if x in collect(keys(m2))
            append!(mb,[x])
        end
    end
    return mb, m1, m2
end

MomBasis (generic function with 1 method)

In [95]:
using BenchmarkTools
N1 = 2
N2 = 3

n1 = 4
n2 = 3

tag = 167

println(cyclebits(tag, n1, n2, N1, N2))
println(cyclebits_opt(tag, n1, n2, N1, N2))

167
167


In [97]:
k1 = 0
k2 = 0
@benchmark MomBasis(k1, k2, N1, N2)

BenchmarkTools.Trial: 
  memory estimate:  5.06 MiB
  allocs estimate:  40948
  --------------
  minimum time:     7.558 ms (0.00% GC)
  median time:      8.131 ms (0.00% GC)
  mean time:        8.348 ms (3.36% GC)
  maximum time:     49.005 ms (82.71% GC)
  --------------
  samples:          599
  evals/sample:     1